In [1]:
# ---------------- For testing only : comment afterwards
# import os

# # Set the root directory
# root_directory = r"C:\Users\HP\Desktop\github-aditya0by0\python-chebai"
# os.chdir(root_directory)

# # Verify the current working directory
# print("Current working directory:", os.getcwd())

Current working directory: C:\Users\HP\Desktop\github-aditya0by0\python-chebai


In [2]:
import pandas as pd

from chebai.result.utils import (
    evaluate_model,
    load_results_from_buffer,
)
from chebai.result.classification import print_metrics
from chebai.models.electra import Electra
from chebai.preprocessing.datasets.chebi import ChEBIOver50, ChEBIOver100
import os
import tqdm
import torch
import pickle

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

C:\Users\HP\anaconda3\envs\env_chebai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


In [3]:
# specify the checkpoint name
checkpoint_name = "my_trained_model"
checkpoint_path = os.path.join("logs", f"{checkpoint_name}.ckpt")
kind = "test"  # replace with "train" / "validation" to run on train / validation sets
buffer_dir = os.path.join("results_buffer", checkpoint_name, kind)
# make sure to use the same data module and model class that were used during training
data_module = ChEBIOver50(
    chebi_version=231,
)
# load chebi data if missing and perform dynamic splits
data_module.prepare_data()
data_module.setup()

model_class = Electra

Check for processed data in data\chebi_v231\ChEBI50\processed\smiles_token
Cross-validation enabled: False


Check for processed data in data\chebi_v231\ChEBI50\processed
saving 771 tokens to C:\Users\HP\Desktop\github-aditya0by0\python-chebai\chebai\preprocessing\bin\smiles_token\tokens.txt...
first 10 tokens: ['[*-]', '[Al-]', '[F-]', '.', '[H]', '[N]', '(', ')', '[Ag+]', 'C']
Get test data split
Split dataset into train / val with given test set


In [4]:
# --------------- For testing only : comment afterwards
# data_module.data_limit = 100
# main_directory = r"C:\Users\HP\Desktop\github-aditya0by0\python-chebai"
# checkpoint_name = r"logs\chebi50_bce_unweighted\version_27\checkpoints\per_epoch=99_val_loss=0.1377_val_macro-f1=0.0200_val_micro-f1=0.2947"
# checkpoint_path = os.path.join(main_directory, f"{checkpoint_name}.ckpt")
# print(checkpoint_path)

C:\Users\HP\Desktop\github-aditya0by0\python-chebai\logs\chebi50_bce_unweighted\version_27\checkpoints\per_epoch=99_val_loss=0.1377_val_macro-f1=0.0200_val_micro-f1=0.2947.ckpt


In [5]:
# evaluates model, stores results in buffer_dir
model = model_class.load_from_checkpoint(checkpoint_path)
if buffer_dir is None:
    preds, labels = evaluate_model(
        model,
        data_module,
        buffer_dir=buffer_dir,
        # No need to provide this parameter for Chebi dataset, "kind" parameter should be provided
        # filename=data_module.processed_file_names_dict[kind],
        batch_size=10,
        kind=kind,
    )
else:
    evaluate_model(
        model,
        data_module,
        buffer_dir=buffer_dir,
        # No need to provide this parameter for Chebi dataset, "kind" parameter should be provided
        # filename=data_module.processed_file_names_dict[kind],
        batch_size=10,
        kind=kind,
    )
    # load data from buffer_dir
    preds, labels = load_results_from_buffer(buffer_dir, device=DEVICE)

100%|████████████████████████████████| 10/10 [00:06<00:00,  1.54it/s]


In [6]:
# Load classes from the classes.txt
with open(os.path.join(data_module.processed_dir_main, "classes.txt"), "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [8]:
# output relevant metrics
print_metrics(
    preds,
    labels.to(torch.int),
    DEVICE,
    classes=classes,
    markdown_output=False,
    top_k=10,
)

Macro-F1: 0.290936
Micro-F1: 0.890380
Balanced Accuracy: 0.507610
Macro-Precision: 0.021964
Micro-Precision: 0.908676
Macro-Recall: 0.020987
Micro-Recall: 0.872807
Top 10 classes (F1-score):
1. 23367 - F1: 1.000000
2. 33259 - F1: 1.000000
3. 36914 - F1: 1.000000
4. 24431 - F1: 1.000000
5. 33238 - F1: 1.000000
6. 36357 - F1: 1.000000
7. 37577 - F1: 1.000000
8. 24867 - F1: 1.000000
9. 33579 - F1: 0.974026
10. 24866 - F1: 0.973684
Found 63 classes with F1-score == 0 (and non-zero labels): 17792, 22563, 22632, 22712, 24062, 24834, 25108, 25693, 25697, 25698, 25699, 25806, 26151, 26217, 26218, 26421, 26469, 29347, 32988, 33240, 33256, 33296, 33299, 33304, 33597, 33598, 33635, 33655, 33659, 33661, 33670, 33671, 33836, 33976, 35217, 35273, 35479, 35618, 36364, 36562, 36916, 36962, 36963, 37141, 37143, 37622, 37929, 37960, 38101, 38104, 38166, 38835, 39203, 46850, 47704, 47916, 48592, 50047, 50995, 72544, 79389, 83565, 139358
